In [1]:
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import nearest_point_projection as npp
import interpolation as intp
import ipyvolume as ipv
%matplotlib ipympl

## Circular projection

In [2]:
def equidistant_curve_2D(X1, X2, s):
    x1 = X1 @ intp.lagrange_poly(X1.shape[1] - 1, s)
    xm = np.zeros_like(x1)
    for i in range(len(s)):
        sol = npp.circular_point_projection(
            'Lagrange polynoms', X1, X2, s[i]
        )
        try:
            if sol == False:
                continue
        except:
            xm[:,i] = x1[:,i] + sol[1:3]
    return xm

In [3]:
X1_2D = np.array([
    [0,1],
    [1,2],
    [2,3],
    [2.5,2.5]
]).T
X2_2D = np.array([
    [-1,1],
    [1,0],
    [1.5,2],
    [2.75,2]
]).T

s1 = s2 = np.linspace(-1,1)
x1_2D = X1_2D @ intp.lagrange_poly(3, s1)
x2_2D = X2_2D @ intp.lagrange_poly(3, s2)

res2D = npp.circular_point_projection(
    'Lagrange polynoms', X1_2D, X2_2D, s1[20]
)
x2_sol = (X2_2D @ intp.lagrange_poly(3, [res2D[0]])).flatten()

xm_2D = equidistant_curve_2D(X1_2D, X2_2D, s1)

No convergence


In [4]:
plt.plot(x1_2D[0], x1_2D[1])
plt.plot(x2_2D[0], x2_2D[1])
plt.plot(x1_2D[0,20], x1_2D[1,20], 'o')
plt.plot(x2_sol[0], x2_sol[1], 'o')
plt.plot([x1_2D[0,20], x1_2D[0,20]+res2D[1]], [x1_2D[1,20], x1_2D[1,20]+res2D[2]])
plt.plot([x2_sol[0], x2_sol[0]+res2D[3]], [x2_sol[1], x2_sol[1]+res2D[4]])

plt.plot(xm_2D[0], xm_2D[1], '.')

plt.show()

FigureCanvasNbAgg()

In [1]:
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import nearest_point_projection as npp
import interpolation as intp
import ipyvolume as ipv
%matplotlib ipympl

## Spherical projection

In [2]:
X1 = np.array([
    [0,-0.3,0.5],
    [1,0.1,0.6],
    [2,-0.1,0.7]
]).T
X2 = np.array([
    [0.5,-1,-0.1],
    [1.5,0,0.1],
    [2.5,1,-0.1]
]).T
print("Problem of objectivity (the same point on one curve should project to the same on the other.")
print("All constraints are fullfilled (checked), however, they still don't project to the same point.")
(s20, x20, R10) = npp.spherical_point_projection_3(
    "Lagrange polynoms", X1, X2, -0.2, MAXITER=30, TOLER=1e-15
)
print('\tstart on s1:', -0.2, 'finish s2:', s20)
(s10, x10, R20) = npp.spherical_point_projection_3(
    "Lagrange polynoms", X2, X1, s20, MAXITER=30, TOLER=1e-15
)
print('\tstart on s2:', s20, 'finish s1:', s10)
print("Could be within numerical error, could be the problem with local extremes.")
print("Looking at the numerical errors of the constraints, it is probably the problem of numerical precision.")
print("This is, however, not concerning as the gap is still symmetric.")

Problem of objectivity (the same point on one curve should project to the same on the other.
All constraints are fullfilled (checked), however, they still don't project to the same point.
	start on s1: -0.2 finish s2: -0.3836654988050439
	start on s2: -0.3836654988050439 finish s1: -0.18565400006199306
Could be within numerical error, could be the problem with local extremes.
Looking at the numerical errors of the constraints, it is probably the problem of numerical precision.
This is, however, not concerning as the gap is still symmetric.


---

In [3]:
s20 = npp.spherical_point_projection_3(
    "Lagrange polynoms", X1, X2, 1
)
print(s20)
s10 = npp.spherical_point_projection_3(
    "Lagrange polynoms", X2, X1, s20[0]
)
print(s10)

(0.33609180026239793, array([1.8360918 , 0.3360918 , 0.07740846]), array([-0.12728879, -0.1753149 ,  0.39631341]))
(0.838283245253492, array([ 1.83828325, -0.02698732,  0.68382832]), array([-0.1373793 ,  0.0892549 , -0.35797063]))


In [4]:
(s2array, R1array, dR1array, ddR1array) = npp.spherical_point_projection_test(
    "Lagrange polynoms", X1, X2, 1
)
(s1array, R2array, dR2array, ddR2array) =  npp.spherical_point_projection_test(
    "Lagrange polynoms", X1, X2, s10[0]
)

In [5]:
plt.hlines((0,), -1, 1)
plt.plot(s2array, R1array, 'b')
plt.plot(s1array, R2array, 'r')
plt.plot(s2array, dR1array / R1array, 'b--')
plt.plot(s1array, dR2array / R2array, 'r--')

FigureCanvasNbAgg()

In [6]:
s1 = np.linspace(-1,1,10)
s2 = np.linspace(-1,1,10)
x1 = X1 @ intp.lagrange_poly(X1.shape[1]-1, s1)
x2 = X2 @ intp.lagrange_poly(X2.shape[1]-1, s2)
j = 3

(s20, x20, R10) = npp.spherical_point_projection_3(
    "Lagrange polynoms", X1, X2, s1[j], MAXITER=30, TOLER=1e-15
)

xm = x1[:,j] - R10

u = np.linspace(0, 2 * np.pi, 100)
v = np.linspace(0, np.pi, 100)
x1s = xm[0] + np.linalg.norm(R10) * np.outer(np.cos(u), np.sin(v))
y1s = xm[1] + np.linalg.norm(R10) * np.outer(np.sin(u), np.sin(v))
z1s = xm[2] + np.linalg.norm(R10) * np.outer(np.ones(np.size(u)), np.cos(v))


fig = plt.figure()
ax = plt.axes(projection='3d')

ax.plot_surface(x1s, y1s, z1s, linewidth=0.0, alpha=0.3)

ax.plot(x1[0], x1[1], x1[2], label='(1)')
ax.plot(x2[0], x2[1], x2[2], label='(2)')

ax.plot([x1[0,j]], [x1[1,j]], [x1[2,j]], 'b.')
ax.plot([xm[0]], [xm[1]], [xm[2]], 'b.')
ax.plot([x20[0]], [x20[1]], [x20[2]], 'b.')

ax.set_xlim((-1, 3))
ax.set_ylim((-1, 3))
ax.set_zlim((-1, 3))

plt.legend()

FigureCanvasNbAgg()

---

In [13]:
def equidistant_curve_3D(X1, X2, s):
    x1 = X1 @ intp.lagrange_poly(X1.shape[1] - 1, s)
    s2 = []
    x2 = []
    xm = []
    for i in range(len(s)):
        sol = npp.spherical_point_projection_2(
            "Lagrange polynoms", X1, X2, s[i]
        )
        if sol == False:
            s2.append(100)
            x2.append([0,0,0])
            xm.append([0,0,0])
        else:
            (s20, x20, R10) = sol
            s2.append(s20)
            x2.append(x20)
            xm.append(x1[:,i] - R10)
    return (np.array(s2), np.array(x2).T, np.array(xm).T)

In [14]:
s1 = np.linspace(-1,1,10)
s2 = np.linspace(-1,1,10)
x1 = X1 @ intp.lagrange_poly(X1.shape[1]-1, s1)
x2 = X2 @ intp.lagrange_poly(X2.shape[1]-1, s2)
(s20, x20, xm_3D2) = equidistant_curve_3D(X1, X2, s1)
(s10, x10, xm_3D1) = equidistant_curve_3D(X2, X1, s2)

In [15]:
fig = ipv.figure()
ipv.pylab.plot(x1[0], x1[1], x1[2], color='red')
ipv.pylab.plot(x2[0], x2[1], x2[2], color='blue')
ipv.pylab.plot(xm_3D2[0], xm_3D2[1], xm_3D2[2], color='green')
ipv.pylab.plot(xm_3D1[0], xm_3D1[1], xm_3D1[2], color='orange')

g3 = ipv.scatter(
    np.hstack((
        x1[0].reshape(x1.shape[1],1),
        x20[0].reshape(x1.shape[1],1),
        xm_3D2[0].reshape(x1.shape[1],1)
    )),
    np.hstack((
        x1[1].reshape(x1.shape[1],1),
        x20[1].reshape(x1.shape[1],1),
        xm_3D2[1].reshape(x1.shape[1],1)
    )),
    np.hstack((
        x1[2].reshape(x1.shape[1],1),
        x20[2].reshape(x1.shape[1],1),
        xm_3D2[2].reshape(x1.shape[1],1)
    )),
    marker='sphere', color='green'
)

ipv.animation_control(g3)

ipv.xyzlim(-1,5)

ipv.show()